In [5]:
from tqdm import tqdm
from collections import defaultdict
import os
import json
from openai import AzureOpenAI
from dotenv import load_dotenv
import time
from collections import deque
from threading import Lock
import asyncio

In [ ]:
# Copy RePASs repo to validate our results - ONLY RUN ONCE
#!git clone https://github.com/RegNLP/RePASs.git && cd RePASs

In [21]:
# Cargar variables de entorno
load_dotenv()

True

In [6]:
# Proceso para cargar una colección de pasajes desde el directorio de documentos estructurados
ndocs = 40  # Número de documentos a procesar
passages = defaultdict(str) # Lista para almacenar la colección de pasajes

# Lee cada documento y extrae los pasajes relevantes
for i in range(1, ndocs + 1):
    with open(os.path.join("ObliQADataset/StructuredRegulatoryDocuments", f"{i}.json")) as f:
        doc = json.load(f)  # Carga el contenido del documento JSON
        for psg in doc:  # Recorre cada pasaje del documento
            passages[psg["ID"]] = psg["Passage"]

In [7]:
rankings_dict = defaultdict(list)

# Abrimos el archivo de rankings y lo cargamos en memoria
with open('data/rankings_hybrid.trec', 'r') as f:
    # Formato de línea: QuestionID Q0 DocumentID Rank Score Método
    for line in f:
        parts = line.strip().split()
        question_id = parts[0]
        document_id = parts[2]
        rank = int(parts[3])
        rankings_dict[question_id].append(document_id)

## Standard deployment

In [44]:
endpoint = os.getenv('QNA_ENDPOINT_URL')
openAIKey = os.getenv('QNA_OPENAI_API_KEY')
llm_model = 'gpt-35-turbo'

if not endpoint:
    raise ValueError("No se ha definido la variable de entorno QNA_ENDPOINT_URL")

if not openAIKey:
    raise ValueError("No se ha definido la variable de entorno QNA_OPENAI_API_KEY")

openAI_client = AzureOpenAI(
    azure_endpoint=endpoint,
    api_key=openAIKey,
    api_version="2024-05-01-preview"
)

class Throttle:
    def __init__(self, rate_limit, time_window):
        self.rate_limit = rate_limit # Numero de llamadas permitidas por fraccion de tiempo
        self.time_window = time_window # Fraccion de tiempo en segundos
        self.calls = deque() # Almacena los tiempos de las llamadas en un deque (Array que permite agregar y remover elementos de ambos extremos en O(1))
        self.lock = Lock() # Lock para asegurar que solo un hilo accede a la lista de llamadas a la vez
        self.queue = asyncio.Queue() # Una cola para almacenar las llamadas que se hicieron mientras se estaba esperando e intentar nuevamente

    def __call__(self, func):
        async def wrapped_func(*args, **kwargs):
            # Referenciar la funcion fuera del context local
            nonlocal func
            # Bloquear el acceso a la lista de llamadas
            with self.lock:
                current_time = time.time()
                
                # Remover llamadas que ya no están en la ventana de tiempo
                while self.calls and self.calls[0] < current_time - self.time_window:
                    self.calls.popleft()

                # Si todavia hay espacio en la ventana de tiempo, agregar la llamada actual y ejecutar la función    
                if len(self.calls) < self.rate_limit:
                    self.calls.append(current_time)
                    return await func(*args, **kwargs)
                else:
                    # De lo contrario, agregar la llamada a la cola y esperar
                    await self.queue.put((func, args, kwargs)) # Agregar la llamada a la cola
                    
                    # Procesa las llamadas en la cola
                    while not self.queue.empty():
                        # Sacar la llamada de la cola
                        func, args, kwargs = await self.queue.get()
                        async with self.lock:
                            current_time = time.time()

                            # Remover llamadas que ya no están en la ventana de tiempo
                            while self.calls and self.calls[0] < current_time - self.time_window:
                                self.calls.popleft()
                            
                            # Si ya hay espacio en la ventana de tiempo, ejecutar la función y sacar de la cola
                            if len(self.calls) < self.rate_limit:
                                self.calls.append(current_time)
                                result = await func(*args, **kwargs)
                                self.queue.task_done()
                                return result
                            else:
                                # De lo contrario, esperar 10 segundos antes de intentar nuevamente
                                await asyncio.sleep(10)
                    
        return wrapped_func

# Permite maximo 60 llamadas cada 50 segundos. Las llamadas se van encolando y se ejecutaran a medida que se libere espacio
@Throttle(rate_limit=60, time_window=50)
async def summarize_answer(question: str, relevant_passages: list[str]) -> str:
    """
    Utilizar un modelo LLM para crear una respuesta a la pregunta a partir de la lista
    de pasajes relevantes.
    """

    # Construir el prompt para el modelo LLM
    system_prompt = """
You are an expert legal assistant specializing in compliance and regulatory documents, with a particular focus on the regulations and rules issued by Abu Dhabi Global Markets (ADGM). When responding to a query, your primary objective is to provide a **comprehensive**, **accurate**, and **coherent** answer that fully addresses the compliance and obligation requirements of the question. 

### Key Guidelines for Response:

1. **Relevance**:  
   Start with the most relevant passage (Passage 1). This should contain the majority of the information needed to answer the query. If the answer is not fully addressed in Passage 1, proceed to the next relevant passage (Passage 2), and so on, until the query is answered in full. Avoid using information that is not contained in the regulatory content.

2. **Accuracy and Completeness**:  
   Ensure that your response is fully aligned with the compliance requirements, rules, and obligations outlined in the regulatory passages. If necessary, infer logical conclusions based on the provided text, but do not make up or introduce any information that is not explicitly stated in the provided regulatory content.

3. **Clarity and Coherence**:  
   Your answer should be clear, well-organized, and free of contradictions. Present information in a structured manner, making sure each point is addressed in sequence, referencing specific rules, regulations, or obligations as relevant.

5. **Regulatory References**:  
   Whenever possible, cite the specific regulation, rule, or article number from the relevant passages that supports your answer. This will enhance the accuracy and reliability of your response.

6. **No Contradictions**:  
   Ensure that there are no contradictions in your answer. All information provided should align with the regulatory content from the relevant passages.
    """

    user_prompt = f"Question: {question}\n\nPassages:\n\n"
    for idx, passage in enumerate(relevant_passages, 1):
        user_prompt += f"{idx}. {passage}\n\n"

    completion = openAI_client.chat.completions.create(
        model=llm_model, # Estamos usando el modelo gpt-3.5-turbo
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.3, # Controla la aleatoriedad de las respuestas generadas - Menor valor, respuestas más deterministicas
        frequency_penalty=0.0,
        presence_penalty=0.0,
        stop=None,
        stream=False,
        max_tokens=800,
    )

    return completion.choices[0].message.content
    

In [46]:
answers = []
it = 0

# Abrimos el archivo JSON que contiene las consultas de prueba (ObliQA_test.json)
with open("ObliQADataset/ObliQA_test.json") as f:
    data = json.load(f)  # Cargamos el contenido del archivo JSON
    
    # Iteramos sobre cada entrada (pregunta) en el archivo de datos
    for e in tqdm(data):  # tqdm agrega una barra de progreso durante la iteración
        query = e['Question']  # Extraemos la pregunta o consulta desde el campo 'Question'
        questionId = e["QuestionID"] # Extraemos el ID de la pregunta

        # TODO: Extract more relevant passages up to a given threshold (e.g., 0.7) and consider abrupt 
        # changes in relevance score
        retrieved_passages = [passages[doc] for doc in rankings_dict[questionId][:3]]

        answer = await summarize_answer(query, retrieved_passages)

        answers.append({
            "QuestionID": questionId,
            "RetrievedPassages": retrieved_passages,
            "Answer": answer
        })

# Guardamos las respuestas en un archivo JSON
with open("data/answers.json", "w") as f:
    json.dump(answers, f, indent=2)        

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2786/2786 [1:37:42<00:00,  2.10s/it]


In [ ]:
## Script para evaluar los resultados que se guardaran en /RePASs/data/hybrid
## Se deben correr activando el ambiente virtual definido en RePASs

#python scripts/evaluate_model.py --input_file ./../data/answers.json --group_method_name hybrid